# Draft: STIPS Tutorial

***

## Introduction

**STIPS**, or the Space Telescope Imaging Product Simulator, is a tool developed by the Space Telescope Science Institute for simulating observations with the full array of detectors on the Roman Wide Field Imager (WFI).

Though it trades some accuracy in order to capture the full array of detectors simulations – see the Pandeia Tutorial for higher accuracy simulations of smaller areas – STIPS simulations do take Roman's exposure-level pipeline ("Level 2") into account. This means generated scenes come with readouts of many calibration residuals. Scenes are also returned with Poisson and readout noise estimates and can incorporate instrumental distortion.

This notebook is a starter guide to simulating and manipulating scenes with STIPS. STIPS requires separate reference data both for itself and for some of its dependencies, so be sure you've followed the complete installation instructions before attempting to run the notebook.

## Imports

Besides the STIPS-related imports, `astropy.io.fits` will help write a FITS table on the fly.

In [ ]:
import stips

from astropy.io import fits
from stips.observation_module import ObservationModule
from stips.scene_module import SceneModule

# USE PYTHON 3.11
# GET EXAMPLE FILE IN LAST TUTORIAL (PSFs + adding sources)

### Environment report

To verify the existing STIPS installation alongside its associated data files and dependencies, run the cell below.

[SAY SHOULD MATCH https://stips.readthedocs.io/en/latest/installation.html#stips-requirements ??]

In [ ]:
print(stips.__env__report__)

***

## Examples

* Installation test (overview) -- included in Intro/Import section
* Basic STIPS usage (tutorial)
* Generating scene from user-based catalog (tutorial)
* Modifying observation parameters (tutorial)
* PSFs and adding sources

### Basic STIPS usage

In [ ]:
# Build observation parameters
obs = {'instrument': 'WFI', 'filters': ['F129'], 'detectors': 1,
       'background': 'pandeia', 'observations_id': 42, 'exptime': 300,
       'offsets': [{'offset_id': 1, 'offset_centre': False, 'offset_ra': 0.0,
                    'offset_dec': 0.0, 'offset_pa': 0.0}]
      }

In [ ]:
# Create observation object
obm = ObservationModule(obs, ra=90, dec=30, pa=0, seed=42, cores=6)

In [1]:
'#' * 80

'################################################################################'

In [7]:
rm ../webbpsf_tutorial/webbpsf_tutorial.ipynb ../pandeia_tutorial/pandeia_tutorial.ipynb

#### Create a simple astronomical scene

* `id`: Object ID
* `ra`: Right ascension (RA), in degrees
* `dec`: Declination (DEC), in degrees
* `flux`: Flux, in `units` (defined below)
* `type`: Type of PROFILE(???) (`'point'` or `'sersic'`)
* `n`: Sersic profile index (in what system???)
* `re`: Half-light radius, in pixels
* `phi`: Position angle of major axis, in degrees
* `ratio`: Major and minor axis ratio
* `notes`: Per-source comments
* `units`: ???

In [ ]:
from astropy.io import fits
 
cols = []
cols.append(fits.Column(name='id', array=[1, 2], format='K'))
cols.append(fits.Column(name='ra', array=[90.02, 90.03], format='E'))
cols.append(fits.Column(name='dec', array=[29.98, 29.97], format='D'))
cols.append(fits.Column(name='flux', array=[0.00023, 0.0004], format='D'))
cols.append(fits.Column(name='type', array=['point', 'point'], format='8A'))
cols.append(fits.Column(name='n', array=[0, 0], format='D'))
cols.append(fits.Column(name='re', array=[0, 0], format='D'))
cols.append(fits.Column(name='phi', array=[0, 0], format='D'))
cols.append(fits.Column(name='ratio', array=[0, 0], format='D'))
cols.append(fits.Column(name='notes', array=['', ''], format='8A'))
cols.append(fits.Column(name='units', array=['j', 'j'], format='8A'))
 
# Create output fits table
hdut = fits.BinTableHDU.from_columns(cols)
hdut.header['TYPE'] = 'mixed'
hdut.header['FILTER'] = 'F129'
 
# Write to disk
cat_file = 'catalog.fits'
hdut.writeto(cat_file, overwrite=True)

#### Simulate an image

In [ ]:
# Initialize the local instrument
obm.nextObservation()
 
# Add catalog with sources
cat_name = obm.addCatalogue(cat_file)
 
# Add error to image
psf_file = obm.addError()

In [ ]:
# Call the final method
fits_file, mosaic_file, params = obm.finalize(mosaic=False)
print(f"Output FITS file is {fits_file}")

In [ ]:
# DISPLAY IMAGE

### Generate scene from a user-created catalog

In [ ]:
# USE SAME OBS PARAMS FOR BOTH CATALOGS
obs_prefix = 'notebook_example'
obs_ra = 150.0
obs_dec = -2.5

In [ ]:
# STELLAR
scm_stellar = SceneModule(out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec)
 
stellar_parameters = {
                      'n_stars': 100,
                      'age_low': 7.5e12,
                      'age_high': 7.5e12,
                      'z_low': -2.0,
                      'z_high': -2.0,
                      'imf': 'salpeter',
                      'alpha': -2.35,
                      'binary_fraction': 0.1,
                      'clustered': True,
                      'distribution': 'invpow',
                      'radius': 100.0,
                      'radius_units': 'pc',
                      'distance_low': 20.0,
                      'distance_high': 20.0,
                      'offset_ra': 0.0,
                      'offset_dec': 0.0
                     }
 
stellar_cat_file = scm_stellar.CreatePopulation(stellar_parameters)
print(f"Stellar population saved to file {stellar_cat_file}")

In [ ]:
# GALACTIC
scm_galactic = SceneModule(out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec)
  
galaxy_parameters = {
                     'n_gals': 10,
                     'z_low': 0.0,
                     'z_high': 0.2,
                     'rad_low': 0.01,
                     'rad_high': 2.0,
                     'sb_v_low': 30.0,
                     'sb_v_high': 25.0,
                     'distribution': 'uniform',
                     'clustered': False,
                     'radius': 200.0,
                     'radius_units': 'arcsec',
                     'offset_ra': 0.0,
                     'offset_dec': 0.0,
                    }
 
galaxy_cat_file = scm_galactic.CreateGalaxies(galaxy_parameters)
print(f"Galaxy population saved to file {galaxy_cat_file}")

In [ ]:
# HOW DOES THIS RELATE TO OTHERS?
offset_1 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 2.0,
          'offset_dec': 0.0,
          'offset_pa': 0.5
          }
 
residuals_1 = {
             'residual_flat': True,
             'residual_dark': True,
             'residual_cosmic': False,
             'residual_poisson': False,
             'residual_readnoise': False
            }
 
observation_parameters_1 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 3,
                          'distortion': True,
                          'background': 0.24,
                          'observations_id': 1,
                          'exptime': 1500,
                          'offsets': [offset]
                          }
 
obm_1 = ObservationModule(observation_parameters_1, out_prefix=obs_prefix,
                          ra=obs_ra, dec=obs_dec, residual=residuals_1)
 
obm_1.nextObservation()

### Modify observation parameters

In [ ]:
offset_2 = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 10.0,
          'offset_dec': 0.0,
          'offset_pa': 27
          }
 
residuals_2 = {
             'residual_flat': True,
             'residual_dark': False,
             'residual_cosmic': False,
             'residual_poisson': False,
             'residual_readnoise': True
            }
 
observation_parameters_2 = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 3,
                          'distortion': True,
                          'background': 0.24,
                          'observations_id': 1,
                          'exptime': 1500,
                          'offsets': [offset]
                          }
 
obm_2 = ObservationModule(observation_parameters_2, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec, residual=residuals_2)
 
obm_2.nextObservation()

### Add artifical point source to an observation

In [ ]:
with fits.open('psf_WFI_2.0.0_F129_sca01.fits') as hdul:
    test_psf = stips.utilities.makePSF.make_epsf(hdul[0].data)

In [ ]:
# VARS SEEM DIFFERENT – DOES THIS NEED CODE FROM PREVIOUS EXAMPLES? IF NOT, RDOX REFERENCE
# TO COPYING EXAMPLE 2 CODE IS WRONG.

In [ ]:
offset = {
          'offset_id': 1,
          'offset_centre': False,
          'offset_ra': 0.0,
          'offset_dec': 0.0,
          'offset_pa': 0.0
         }
 
observation_parameters = {
                          'instrument': 'WFI',
                          'filters': ['F129'],
                          'detectors': 1,
                          'distortion': False,
                          'background': 0.15,
                          'observations_id': 1,
                          'exptime': 1000,
                          'offsets': [offset]
                         }
 
obm = ObservationModule(observation_parameters, out_prefix=obs_prefix, ra=obs_ra, dec=obs_dec)
 
# nextObservation is called to move between different combinations of offset and filter.
# It must be called once in order to initialize the observation module to the first observation before adding catalogs.
obm.nextObservation()
 
output_stellar_catalogs = obm.addCatalogue(stellar_cat_file)
output_galaxy_catalogs = obm.addCatalogue(galaxy_cat_file)
 
print(f"Output Catalogs are {output_stellar_catalogs} and {output_galaxy_catalogs}.")
 
psf_file = obm.addError()
 
print("PSF File is {psf_file}")
 
fits_file, mosaic_file, params = obm.finalize(mosaic=False)
 
print(f"Output FITS file is {fits_file}")
print(f"Output Mosaic File is {mosaic_file}")
print(f"Observation Parameters are {params}")

## Aditional Resources
While this isn't always necessary, sometimes you want to provide some more resources for the reader who wants to learn something beyond what's in the notebook. Sometimes these don't exist, but if they do, it's good to put them at the end to give the reader somewhere else to go. Usually a list of links using markdown bullet-plus-link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)

## About this notebook
Let the world know who the author of this great notebook is! If possible/appropriate, include a contact email address for users who might need support (e.g. archive@stsci.edu)

**Author:** Justin Otor, Staff Scientist II.  
**Updated In:** 2024-05

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 